# Qwen 0.5b on GRPO

---
이 노트북은 [will brown](https://x.com/willccbb)이 작성한 [GRPO 데모](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb)의 변형 버전으로, gsm8k 수학 데이터셋을 사용하여 llama-1b를 학습시키는 내용입니다.

우리는 Colab에서 더 잘 작동하도록 다음과 같은 변경사항들을 구현했습니다:
* llama-1b를 Qwen-0.5b로 교체
* vllm을 사용한 생성 방식 도입(상당한 속도 향상). Qwen의 작은 크기 덕분에 GRPO에 사용되는 것과 동일한 GPU에서 vllm을 실행할 수 있습니다
* flash-attn 제거 (Qwen 모델링에서 반복적인 버그 발생, 원인 불명확)

---
Ref:
- [Original Notebook](https://colab.research.google.com/drive/1bfhs1FMLW3FGa8ydvkOZyBNxLYOu0Hev?usp=sharing#scrollTo=Q7qTZbUcg5VD)
---


## 1. 환경 설정
- 시작을 위해서 여기 가이드 보세요: [Setup Guide](../setup/README.md)
- 이후에 아래 셀 실행을 통해서, 필요한 패키지가 설치 되었는지를 확인 합니다.

In [2]:
! pip list | grep -E "sagemaker|vllm|trl|datasets"

datasets                          3.2.0
sagemaker                         2.239.0
sagemaker-core                    1.0.21
trl                               0.14.0
vllm                              0.7.2


## System Prompt , COT Format 정의

In [3]:
import re
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOConfig, GRPOTrainer

# Load and prep dataset

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

/home/sagemaker-user/.conda/envs/grpo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 02-09 01:15:40 __init__.py:190] Automatically detected platform cuda.


2025-02-09 01:15:40,889	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## 데이터 셋 준비

In [4]:
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

## 리워드 정의

In [5]:
# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

## 훈련 파라미터 정의
- 모델 정의
- 훈련 파라미터

In [6]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

output_dir="outputs/Qwen-0.5B-GRPO"
run_name="Qwen-0.5B-GRPO-gsm8k"

training_args = GRPOConfig(
    output_dir=output_dir,
    run_name=run_name,
    learning_rate=5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type='cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_generations=16,
    max_prompt_length=256,
    max_completion_length=200,
    num_train_epochs=1,
    save_steps=100,
    max_grad_norm=0.1,
    log_on_each_node=False,
    use_vllm=True,
    vllm_gpu_memory_utilization=.3,
    vllm_device="cuda:0",
    report_to="none" #I'm disabling Wandb.
)



## 모델 로딩, 토큰나이저 로딩

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=None
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

## S3 Root 폴더 정의

In [8]:
import sagemaker

sm_session = sagemaker.Session()

s3_root_folder = f"s3://{sm_session.default_bucket()}/deepseek/qrpo"


/home/sagemaker-user/.conda/envs/grpo/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## 리모트르로 SageMaker Training Job 실행

In [11]:
from sagemaker.remote_function import remote

@remote(instance_type="ml.p4d.24xlarge", dependencies='./requirements.txt', keep_alive_period_in_seconds=1200)
def wrap_hf_trainer():
    trainer = GRPOTrainer(
        model=model,
        processing_class=tokenizer,
        reward_funcs=[
            xmlcount_reward_func,
            soft_format_reward_func,
            strict_format_reward_func,
            int_reward_func,
            correctness_reward_func],
        args=training_args,
        train_dataset=dataset,
        #peft_config=peft_config
    )
    trainer.train()

[02/09/25 01:51:29] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=316759;file:///home/sagemaker-user/.conda/envs/grpo/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=8670;file:///home/sagemaker-user/.conda/envs/grpo/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

In [12]:
wrap_hf_trainer()

2025-02-09 01:51:37,818 sagemaker.remote_function INFO     Serializing function code to s3://sagemaker-us-east-1-057716757052/wrap-hf-trainer-2025-02-09-01-51-37-818/function


2025-02-09 01:51:47,774 sagemaker.remote_function INFO     Serializing function arguments to s3://sagemaker-us-east-1-057716757052/wrap-hf-trainer-2025-02-09-01-51-37-818/arguments
2025-02-09 01:51:47,964 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmpzarbake8/temp_workspace/sagemaker_remote_function_workspace/requirements.txt'
2025-02-09 01:51:47,965 sagemaker.remote_function INFO     Successfully created workdir archive at '/tmp/tmpzarbake8/workspace.zip'
2025-02-09 01:51:47,992 sagemaker.remote_function INFO     Successfully uploaded workdir to 's3://sagemaker-us-east-1-057716757052/wrap-hf-trainer-2025-02-09-01-51-37-818/sm_rf_user_ws/workspace.zip'
2025-02-09 01:51:47,993 sagemaker.remote_function INFO     Creating job: wrap-hf-trainer-2025-02-09-01-51-37-818


2025-02-09 01:51:52 Starting - Found matching resource for reuse
2025-02-09 01:51:52 Downloading - Downloading input data...
2025-02-09 01:52:03 Training - Training image download completed. Training in progress...INFO: CONDA_PKGS_DIRS is set to '/opt/ml/sagemaker/warmpoolcache/sm_remotefunction_user_dependencies_cache/conda/pkgs'
INFO: PIP_CACHE_DIR is set to '/opt/ml/sagemaker/warmpoolcache/sm_remotefunction_user_dependencies_cache/pip'
INFO: /opt/ml/input/config/resourceconfig.json:
{"current_host":"algo-1","current_instance_type":"ml.p4d.24xlarge","current_group_name":"homogeneousCluster","hosts":["algo-1"],"instance_groups":[{"instance_group_name":"homogeneousCluster","instance_type":"ml.p4d.24xlarge","hosts":["algo-1"]}],"network_interface_name":"eth0"}INFO: Bootstraping runtime environment.
2025-02-09 01:52:29,628 sagemaker.remote_function INFO     Arguments:
2025-02-09 01:52:29,628 sagemaker.remote_function INFO     job_conda_env=None
2025-02-09 01:52:29,628 sagemaker.remote_fu

[02/09/25 01:55:37] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=47282;file:///home/sagemaker-user/.conda/envs/grpo/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=914488;file:///home/sagemaker-user/.conda/envs/grpo/lib/python3.10/site-packages/botocore/httpchecksum.py#487\487]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

                    INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=852177;file:///home/sagemaker-user/.conda/envs/grpo/lib/python3.10/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=280042;file:///home/sagemaker-user/.conda/envs/grpo/lib/python3.10/site-packages/botocore/httpchecksum.py#487\487]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.10/site-packages/sagemaker/remote_function/core/serialization.py:141 in   │
│ deserialize                                                                                      │
│                                                                                                  │
│   138 │   │   """                                                                                │
│   139 │   │                                                                                      │
│   140 │   │   try:                                                                               │
│ ❱ 141 │   │   │   return cloudpickle.loads(bytes_to_deserialize)                                 │
│   142 │   │   except Exception as e:                                                             │
│   143 │   │   │   raise DeserializationError(                                                    │
│   144 │   │   │   │   "Error when deserializing bytes downloaded from {}: {}. "                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: Can't get attribute '_function_setstate' on <module 'cloudpickle.cloudpickle' from 
'/opt/conda/lib/python3.10/site-packages/cloudpickle/cloudpickle.py'>

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 wrap_hf_trainer()                                                                            │
│   2                                                                                              │
│                                                                                                  │
│ /home/sagemaker-user/.conda/envs/grpo/lib/python3.10/site-packages/sagemaker/remote_function/cli │
│ ent.py:387 in wrapper                                                                            │
│                                                                                                  │
│    384 │   │   │   │   │   │   │   )                                                             │
│    385 │   │   │   │   │   │   raise serr                                                        │
│    386 │   │   │   │   │                                                                         │
│ ❱  387 │   │   │   │   │   raise exception                                                       │
│    388 │   │   │   │                                                                             │
│    389 │   │   │   │   raise TimeoutError(                                                       │
│    390 │   │   │   │   │   "Job for remote function timed out before reaching a termination sta  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/sagemaker/remote_function/invoke_function.py:144 in main │
│                                                                                                  │
│   141 │   │   hmac_key = os.getenv("REMOTE_FUNCTION_SECRET_KEY")                                 │
│   142 │   │                                                                                      │
│   143 │   │   sagemaker_session = _get_sagemaker_session(region)                                 │
│ ❱ 144 │   │   _execute_remote_function(                                                          │
│   145 │   │   │   sagemaker_session=sagemaker_session,                                           │
│   146 │   │   │   s3_base_uri=s3_base_uri,                                                       │
│   147 │   │   │   s3